In [1]:
import pandas as pd
import yaml
from pathlib import Path
from datetime import timedelta

In [11]:
# Read YAML file and parse into Python dictionary 
def load_yaml_config(yml_path):
    with open(Path(yml_path), "r") as f:
        config = yaml.safe_load(f)
    return config

# Load a CSV and filter to internal medicine providers
def load_and_filter_csv(csv_path, provider_list):
    df = pd.read_csv(Path(csv_path))
    
    # Convert to date time
    df['date'] = pd.to_datetime(df['date'])
    
    # Filter for internal medicine providers 
    df = df[df['provider'].isin(provider_list)].copy()
    return df

# Add desired number of inpatient days
def expand_inpatient(inpatient_df, length):
    expanded = []
    for _, row in inpatient_df.iterrows():
        start = row['date']
        for i in range(length):
            expanded.append({
                'provider': row['provider'],
                'date': start + timedelta(days = i),
                'type': 'inpatient',
            })
    return pd.DataFrame(expanded)

# Main function to parse all inputs
def parse_inputs(yml_path, inpatient_csv_path, leave_request_csv_path):
    config = load_yaml_config(yml_path)
    providers = list(config['providers'].keys())
    
    # Defaults to 7
    inpatient_length = (
        config.get('clinic_rules', {})
              .get('inpatient_schedule', {})
              .get('inpatient_length', 7)
    )

    inpatient_df = load_and_filter_csv(inpatient_csv_path, providers)
    leave_df = load_and_filter_csv(leave_request_csv_path, providers)

    inpatient_expanded = expand_inpatient(inpatient_df, inpatient_length)

    return config, inpatient_expanded, leave_df

In [12]:
parse_inputs('../config/internal_medicine.yml',
             '../data/inpatient.csv', 
             '../data/leave_requests.csv')

({'clinic_rules': {'clinic_days': ['Monday',
    'Tuesday',
    'Wednesday',
    'Thursday',
    'Friday'],
   'clinic_sessions': {'Monday': ['morning', 'afternoon'],
    'Tuesday': ['morning', 'afternoon'],
    'Wednesday': ['morning', 'afternoon'],
    'Thursday': ['afternoon'],
    'Friday': ['morning', 'afternoon']},
   'random_day_off': {'enabled': True,
    'eligible_days': ['Monday', 'Tuesday', 'Wednesday', 'Friday'],
    'substitute_if_holiday': True,
    'substitute_if_leave': True,
    'override_roles': ['NP', 'PA']},
   'holiday_week_rdo_followup': {'enabled': True,
    'day_options': ['Monday', 'Friday'],
    'applies_to_roles': ['MD', 'DO']},
   'inpatient_schedule': {'inpatient_length': 7,
    'inpatient_starts_on': 'Tuesday',
    'inpatient_ends_on': 'Monday',
    'pre_inpatient_rdo': 'Monday',
    'post_inpatient_rdo': 'Friday'},
   'staffing': {'min_providers_per_session': 1,
    'max_providers_per_session': 4},
   'holiday_dates': [datetime.date(2025, 1, 1),
    datet

In [7]:
yml_dic = load_yaml_config('../config/internal_medicine.yml')

In [ ]:
clinic_section = config.get('clinic_rules', config.get('clinic', {}))
    inpatient_length = clinic_section.get('inpatient_schedule', {}).get('inpatient_length', 7)


In [10]:
yml_dic.get('clinic_rules').get('inpatient_schedule', {}).get('inpatient_length', 7)

7